In [9]:
import requests
import hashlib
import hmac
import time
import pandas as pd 

API_KEY = "C8Lw6mJh4CNYQXVIgdRAv64S5bQzh1RyNBQJNL3C2roe8rsxyTtN8EfB4faadhD3"
API_SECRET = "aokTvHmhUOFnhOlSNp2a7VPh1NVfkydrFHgdqRArKjuoz2AXYJTBOsNsyxhPsXs8"

"C8Lw6mJh4CNYQXVIgdRAv64S5bQzh1RyNBQJNL3C2roe8rsxyTtN8EfB4faadhD3"
"aokTvHmhUOFnhOlSNp2a7VPh1NVfkydrFHgdqRArKjuoz2AXYJTBOsNsyxhPsXs8"

def create_signature(query_string, secret):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def binance_request(endpoint, params):
    base_url = 'https://fapi.binance.com'
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = create_signature(query_string, API_SECRET)
    url = f"{base_url}{endpoint}?{query_string}&signature={signature}"
    headers = {'X-MBX-APIKEY': API_KEY}
    response = requests.get(url, headers=headers)
    return response.json()

def binance_post_request(endpoint, params):
    base_url = 'https://fapi.binance.com'
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = create_signature(query_string, API_SECRET)
    url = f"{base_url}{endpoint}?{query_string}&signature={signature}"
    headers = {'X-MBX-APIKEY': API_KEY}
    response = requests.post(url, headers=headers)
    return response.json()

def get_positions():
    timestamp = int(time.time() * 1000)
    params = {
        'timestamp': timestamp,
        'recvWindow': 5000
    }
    return binance_request('/fapi/v2/positionRisk', params)

def get_history(symbol=None, start_time=None, end_time=None):
    timestamp = int(time.time() * 1000)
    params = {
        'timestamp': timestamp,
        'recvWindow': 5000
    }
    if symbol:
        params['symbol'] = symbol
    if start_time:
        params['startTime'] = start_time
    if end_time:
        params['endTime'] = end_time
    return binance_request('/fapi/v1/allOrders', params)

def get_pnl():
    timestamp = int(time.time() * 1000)
    params = {
        'timestamp': timestamp,
        'recvWindow': 5000
    }
    return binance_request('/fapi/v2/account', params)

def get_price(symbol):
    timestamp = int(time.time() * 1000)
    params = {
        'symbol': symbol,
        'timestamp': timestamp,
        'recvWindow': 5000
    }
    return binance_request('/fapi/v1/ticker/price', params)

def place_order(symbol, side, order_type, quantity, leverage, price=None):
    timestamp = int(time.time() * 1000)
    params = {
        'symbol': symbol,
        'side': side,
        'type': order_type,
        'quantity': quantity,
        'leverage': leverage,
        'timestamp': timestamp,
        'recvWindow': 5000
    }
    if price:
        params['price'] = price
        params['timeInForce'] = 'GTC'  # Good Till Cancelled
    return binance_post_request('/fapi/v1/order', params)

def get_all_symbols():
    response = requests.get('https://api.binance.com/api/v3/exchangeInfo')
    data = response.json()
    symbols = [symbol['symbol'] for symbol in data['symbols']]
    return symbols


In [11]:
asset_df = pd.DataFrame(get_pnl()["assets"])
asset_df

,asset,walletBalance,unrealizedProfit,marginBalance,maintMargin,initialMargin,positionInitialMargin,openOrderInitialMargin,maxWithdrawAmount,crossWalletBalance,crossUnPnl,availableBalance,marginAvailable,updateTime
0,FDUSD,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
1,BTC,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
2,XRP,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
3,TUSD,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
4,BNB,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
5,ETH,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
6,USDT,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0
7,USDC,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,True,0


In [12]:
pos = get_positions()

pos_df = pd.DataFrame(pos)
pos_df

,symbol,positionAmt,entryPrice,breakEvenPrice,markPrice,unRealizedProfit,liquidationPrice,leverage,maxNotionalValue,marginType,isolatedMargin,isAutoAddMargin,positionSide,notional,isolatedWallet,updateTime,isolated,adlQuantile
0,SNTUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,25000,cross,0.00000000,false,BOTH,0,0,0,False,0
1,SUSHIUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,50000,cross,0.00000000,false,BOTH,0,0,0,False,0
2,INJUSDT,0.0,0.0,0.0,0.00000000,0.00000000,0,20,50000,cross,0.00000000,false,BOTH,0,0,0,False,0
3,BNTUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,25000,cross,0.00000000,false,BOTH,0,0,0,False,0
4,STRAXUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,0,cross,0.00000000,false,BOTH,0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,ILVUSDT,0.0,0.0,0.0,0.00000000,0.00000000,0,20,25000,cross,0.00000000,false,BOTH,0,0,0,False,0
304,DUSKUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,25000,cross,0.00000000,false,BOTH,0,0,0,False,0
305,KASUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,25000,cross,0.00000000,false,BOTH,0,0,0,False,0
306,ACHUSDT,0,0.0,0.0,0.00000000,0.00000000,0,20,5000,cross,0.00000000,false,BOTH,0,0,0,False,0


### Étape 4: Initialiser la base de données

**app.py**

In [ ]:
e e e e e e

# Agent : Learning

## Backtest

In [ ]:

from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2022"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

n_SESSION = 100

AGENT_PARAMS = [
    [(1, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(2, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(3, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(4, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(5, "BTC"), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

STRATEGIES = [
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4})
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("MOMENTUM", 5, {"floor" : 4}),
    ("MOMENTUM", 6, {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("MOMENTUM", 9, {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


In [ ]:
AGENT[0].view_report()

In [ ]:
AGENT[0].session.rets_dist.keys()

In [ ]:
AGENT[0].get_report()

In [ ]:
session = AGENT[0].post_event.sessionData
session

In [ ]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2021"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

AGENT_PARAMS = [
    [(1, SYMBOL), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 6, {"floor" : 4})
]


AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


In [ ]:
AGENT[0].view_report()

In [ ]:
pp = AGENT[0].env.post_event.portfolioData
pp.head()

In [ ]:
dd = AGENT[0].env.post_event.tradeData
dd

In [ ]:
import ast

serie = dd["pnl_pct"].values
serie = dd[dd["state"].apply(lambda x : ast.literal_eval(x)[0]) == 'Close']["pnl_pct"].values

In [ ]:
import ast

dd["state"].apply(lambda x : ast.literal_eval(x)[1]).iloc[-1]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Boxplots
fig_box = go.Figure()
fig_box.add_trace(go.Box(y=serie_pos, name='Positives', marker_color='blue'))
fig_box.add_trace(go.Box(y=serie_neg, name='Négatives', marker_color='red'))
fig_box.update_layout(title_text='Boxplots des valeurs positives et négatives', yaxis_title='Valeur')

fig_box.update_layout(height = 800 , width = 1000,
                          legend = dict(orientation="h",
                                        yanchor="bottom", y=1,
                                        xanchor="right", x=0.5),
                          margin = {'t':0, 'b':0, 'l':10, 'r':0}
                          )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Exemple de série de données
data = serie  # Génère 1000 valeurs aléatoires selon une distribution normale

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Afficher les histogrammes
plt.figure(figsize=(12, 6))

# Histogramme des valeurs positives
plt.subplot(1, 2, 1)
plt.hist(positive_data, bins=30, color='blue', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs positives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

# Histogramme des valeurs négatives
plt.subplot(1, 2, 2)
plt.hist(negative_data, bins=30, color='red', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs négatives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

plt.tight_layout()
plt.show()

# Afficher les KDE
plt.figure(figsize=(12, 6))

# KDE des valeurs positives
sns.kdeplot(positive_data, shade=True, color='blue', label='Positives')
# KDE des valeurs négatives
sns.kdeplot(negative_data, shade=True, color='red', label='Négatives')

plt.title('Estimation de la densité de probabilité')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = serie

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Création de la figure et des sous-graphiques
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

# Histogramme et KDE des valeurs positives
sns.histplot(positive_data, bins=30, kde=True, color='blue', ax=axs[0, 0])
axs[0, 0].set_title('Histogramme et KDE des valeurs positives')
axs[0, 0].set_xlabel('Valeur')
axs[0, 0].set_ylabel('Fréquence/Densité')

# Histogramme et KDE des valeurs négatives
sns.histplot(negative_data, bins=30, kde=True, color='red', ax=axs[0, 1])
axs[0, 1].set_title('Histogramme et KDE des valeurs négatives')
axs[0, 1].set_xlabel('Valeur')
axs[0, 1].set_ylabel('Fréquence/Densité')

# Boxplot des valeurs positives
sns.boxplot(data=positive_data, ax=axs[1, 0], color='blue')
axs[1, 0].set_title('Boxplot des valeurs positives')
axs[1, 0].set_xlabel('Valeur')

# Boxplot des valeurs négatives
sns.boxplot(data=negative_data, ax=axs[1, 1], color='red')
axs[1, 1].set_title('Boxplot des valeurs négatives')
axs[1, 1].set_xlabel('Valeur')

plt.tight_layout()
plt.show()


In [ ]:
port = AGENT[0].env.post_event.portfolioData
port

#### --------------------------------------

# *Multi-Agent* v2(1)

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env, SubEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2022-01-01"
END = "2023-10-01"
INTERVAL = "1d"
CAPITAL = 100

config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)

sub_env_1 = SubEnv(symbol="BTC", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_2 = SubEnv(symbol="ETH", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_3 = SubEnv(symbol="SOL", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_4 = SubEnv(symbol="EGLD", capital=CAPITAL, interval=INTERVAL, start=START, end=END)

# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), sub_env_1],
    [(2, "ETH"), sub_env_2],
    [(3, "SOL"), sub_env_3],
    [(4, "EGLD"), sub_env_4]
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 8),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35))
]

In [ ]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.add_agent(*agent_param)
    master.update_params(*strategie)


master.active_agents()
master.run()

In [ ]:
master.agents

In [ ]:
#master.agents[(1, "BTC")].env.get_viz(1, "BTC")
master.agents[(AGENT_PARAMS[0][0])].env.get_viz(*AGENT_PARAMS[0][0])

In [ ]:
master.agents[(AGENT_PARAMS[1][0])].env.get_viz(*AGENT_PARAMS[1][0])

In [ ]:
master.agents[(AGENT_PARAMS[2][0])].env.get_viz(*AGENT_PARAMS[2][0])

In [ ]:
master.agents[(AGENT_PARAMS[3][0])].env.get_viz(*AGENT_PARAMS[3][0])

In [ ]:
master.agents[(4, "EGLD")].env.journal.portfolioData
#master.agents[(3, "SOL")].env.journal.tradesData

In [ ]:
master.agents

In [ ]:
master.agents[(3, "SOL")].env.journal.portfolioData

# *End*

In [ ]:
import threading

threading.enumerate()

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore



# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:


class ProfitManagement:
    def __init__(self, profit_target_percentage, trailing_stop_percentage):
        self.profit_target_percentage = profit_target_percentage
        self.trailing_stop_percentage = trailing_stop_percentage
        self.entry_price = None
        self.profit_target_price = None
        self.trailing_stop_price = None

    def set_entry_price(self, entry_price):
        self.entry_price = entry_price
        self.profit_target_price = entry_price * (1 + self.profit_target_percentage)
        self.trailing_stop_price = entry_price * (1 - self.trailing_stop_percentage)

    def take_profit(self, current_price):
        if current_price >= self.profit_target_price:
            return True
        return False

    def trailing_stop_loss(self, current_price):
        if current_price >= self.trailing_stop_price:
            self.trailing_stop_price = current_price * (1 - self.trailing_stop_percentage)
        return self.trailing_stop_price

# Exemple d'utilisation
profit_target_percentage = 0.05  # Objectif de profit de 5%
trailing_stop_percentage = 0.02  # Trailing stop-loss de 2%

profit_manager = ProfitManagement(profit_target_percentage, trailing_stop_percentage)
profit_manager.set_entry_price(100)  # Prix d'entrée de la position

# Simulation du mouvement des prix
price_history = [100, 110, 120, 115, 125, 130, 135, 130, 140, 145]

for price in price_history:
    if profit_manager.take_profit(price):
        print(f"Prendre profit au prix {price}")
        break
    else:
        trailing_stop_price = profit_manager.trailing_stop_loss(price)
        print(f"Prix actuel : {price}, Trailing stop-loss : {trailing_stop_price}")


In [ ]:
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
from datetime import datetime
from app.binance_api import get_positions, get_history, get_pnl, place_order, get_price, get_all_symbols
from app.db_utils import log_event, get_all_events

def register_callbacks(app):
    @app.callback(
        Output('positions-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_positions(n):
        positions_data = get_positions()
        if 'positions' in positions_data:
            positions = positions_data['positions']
            df_positions = pd.DataFrame(positions)
            df_positions = df_positions[df_positions['positionAmt'].astype(float) != 0]
            return dbc.Table.from_dataframe(df_positions, striped=True, bordered=True, hover=True)
        return "Erreur lors de la récupération des données des positions."

    @app.callback(
        Output('pnl-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_pnl(n):
        pnl_data = get_pnl()
        if 'assets' in pnl_data:
            assets = pnl_data['assets']
            df_pnl = pd.DataFrame(assets)
            df_pnl = df_pnl[df_pnl['walletBalance'].astype(float) > 0]
            return dbc.Table.from_dataframe(df_pnl, striped=True, bordered=True, hover=True)
        return "Erreur lors de la récupération des données du PnL."

    @app.callback(
        [Output('history-div', 'children'), Output('history-graph', 'figure'), Output('detailed-graph', 'figure')],
        [Input('submit-button', 'n_clicks')],
        [State('date-picker-range', 'start_date'), State('date-picker-range', 'end_date')]
    )
    def update_history(n_clicks, start_date, end_date):
        if n_clicks > 0:
            start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
            end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)
            history_data = get_history(None, start_timestamp, end_timestamp)
            if history_data:
                df_history = pd.DataFrame(history_data)
                table = dbc.Table.from_dataframe(df_history, striped=True, bordered=True, hover=True)

                fig = px.line(df_history, x='time', y='price', color='symbol', title='Prix au Fil du Temps')
                detailed_fig = px.scatter(df_history, x='time', y='price', color='symbol', size='quantity', title='Transactions Individuelles')

                return table, fig, detailed_fig
        return "Sélectionnez une période et cliquez sur 'Afficher l'historique'", {}, {}

    @app.callback(
        Output('assets-div', 'children'),
        [Input('interval-positions-component', 'n_intervals')]
    )
    def update_assets(n):
        positions_data = get_positions()
        if 'positions' in positions_data:
            positions = positions_data['positions']
            df_positions = pd.DataFrame(positions)
            df_positions = df_positions[df_positions['positionAmt'].astype(float) != 0]

            df_positions = df_positions.head(3)

            figures = []
            for asset in df_positions['symbol']:
                start_timestamp = int(pd.Timestamp('today') - pd.DateOffset(days=30).timestamp() * 1000)
                end_timestamp = int(pd.Timestamp('today').timestamp() * 1000)
                history_data = get_history(asset, start_timestamp, end_timestamp)
                if history_data:
                    df_history = pd.DataFrame(history_data)
                    fig = px.line(df_history, x='time', y='price', title=f'Tendances de {asset}')
                    figures.append(dcc.Graph(figure=fig))

            return figures
        return "Erreur lors de la récupération des données des actifs."

    @app.callback(
        Output('order-preview', 'children'),
        [Input('order-symbol', 'value'), Input('order-quantity', 'value'), Input('order-leverage', 'value')]
    )
    def preview_order(symbol, quantity, leverage):
        if symbol and quantity and leverage:
            price_data = get_price(symbol)
            if 'price' in price_data:
                price = float(price_data['price'])
                total = price * quantity * leverage
                return f"Prix actuel : {price} USD, Total : {total} USD (avec un levier de {leverage}x)"
        return ""

    @app.callback(
        [Output('order-result', 'children'), Output('order-result', 'className')],
        [Input('submit-order', 'n_clicks')],
        [State('order-symbol', 'value'),
         State('order-side', 'value'),
         State('order-type', 'value'),
         State('order-quantity', 'value'),
         State('order-leverage', 'value'),
         State('order-price', 'value')]
    )
    def place_order_callback(n_clicks, symbol, side, order_type, quantity, leverage, price):
        if n_clicks > 0:
            if not symbol or not side or not order_type or not quantity or not leverage or (order_type == 'LIMIT' and not price):
                return "Tous les champs sont obligatoires pour les ordres limit", "alert alert-danger"
            if quantity <= 0 or leverage <= 0 or (price and price <= 0):
                return "La quantité, le levier et le prix doivent être des valeurs positives", "alert alert-danger"
            
            result = place_order(symbol, side, order_type, quantity, leverage, price)
            event_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if 'orderId' in result:
                log_event('ORDER_PLACED', event_time, symbol, side, order_type, quantity, price, 'SUCCESS')
                return f"Ordre placé avec succès : {result}", "alert alert-success"
            else:
                log_event('ORDER_FAILED', event_time, symbol, side, order_type, quantity, price, 'FAILURE')
                return f"Erreur lors du placement de l'ordre : {result}", "alert alert-danger"
        return "", ""

    @app.callback(
        Output('events-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def display_events(n):
        events = get_all_events()
        df_events = pd.DataFrame(events, columns=['ID', 'Type', 'Time', 'Symbol', 'Side', 'Order Type', 'Quantity', 'Price', 'Result'])
        return dbc.Table.from_dataframe(df_events, striped=True, bordered=True, hover=True)

    @app.callback(
        Output('capital-graph', 'figure'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_capital_graph(n):
        pnl_data = get_pnl()
        if 'assets' in pnl_data:
            assets = pnl_data['assets']
            df_pnl = pd.DataFrame(assets)
            df_pnl = df_pnl[df_pnl['walletBalance'].astype(float) > 0]
            fig = px.line(df_pnl, x='asset', y='walletBalance', title='Évolution du Capital')
            return fig
        return {}

    # Callback pour afficher les points d'entrée et de sortie
    @app.callback(
        Output('history-graph', 'figure'),
        [Input('order-symbol', 'value')]
    )
    def update_entry_exit_points(symbol):
        positions_data = get_positions()
        history_data = get_history(symbol)
        if positions_data and history_data:
            df_positions = pd.DataFrame(positions_data['positions'])
            df_history = pd.DataFrame(history_data)
            fig = px.line(df_history, x='time', y='price', title=f'Historique des prix de {symbol}')
            for index, row in df_positions.iterrows():
                if row['symbol'] == symbol:
                    entry_time = row['entryTime']
                    exit_time = row.get('exitTime')
                    fig.add_scatter(x=[entry_time], y=[row['entryPrice']], mode='markers', name='Point d\'entrée')
                    if exit_time:
                        fig.add_scatter(x=[exit_time], y=[row['exitPrice']], mode='markers', name='Point de sortie')
            return fig
        return px.line(title=f'Aucune donnée disponible pour {symbol}')

